In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
import json

In [ ]:
def read_csv():
    """Read normalized datas.
    
        Ruturn:
            The tuple of features and targets.
    """
    features_URL = "/home/yuyouyu/WorkSpace/106Lab_Data/fatigue_prediction/9-16/normalized_data.csv"
    target_URL = "/home/yuyouyu/WorkSpace/106Lab_Data/fatigue_prediction/9-16/targets.csv"
    features, targets = pd.read_csv(features_URL), pd.read_csv(target_URL)
    return features.loc[: , ~features.columns.str.contains("^Unnamed")].values, targets.loc[: , ~targets.columns.str.contains("^Unnamed")].values

In [ ]:
class netModel(nn.Module):
    def __init__(self):
        super(netModel, self).__init__()
        self.linear1 = nn.Linear(16, 32)
        self.linear2 = nn.Linear(32, 16)
        self.linear3 = nn.Linear(16, 8)
        self.linear4 = nn.Linear(8, 1)
        self.initNetParams()

    def forward(self, x):
        out = nn.functional.relu(self.linear1(x))
        out = nn.functional.relu(self.linear2(out))
        out = nn.functional.relu(self.linear3(out))
        out = nn.functional.relu(self.linear4(out))
        return out
    def initNetParams(self):
        '''Init net parameters.'''
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight.data)
                nn.init.constant_(m.bias.data,0.1)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0,0.01)
                m.bias.data.zero_()


In [ ]:
def datas_to_device(device, *datas):
    """Turn numpy datas to GPU/CPU datas.
        
        Argument: 
            device: CPU or GPU device name.
            datas: The data need to be ttransform to device.
        
        Return:
              The data of specified device. 
    """
    device_data = []
    for data in datas:
        device_data.append(torch.from_numpy(data).to(device))
    return device_data

In [ ]:
def dump_json(info):
    with open("/home/yuyouyu/WorkSpace/106Lab_Data/fatigue_prediction/9-16/result.json", "w") as f:
        f.write(json.dumps(info, ensure_ascii=False, indent=4, separators=(',', ':')))

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
features, targets = read_csv()
epoch_num = 5000
target_names = {'屈服强度': 'yield_strength',
               '延伸率': 'elongation',
               '抗拉强度': 'Tensile_strength',
               '断面收缩率':'Reduction_of_section'}
kf = KFold(n_splits=10, shuffle=True, random_state=1)
result = {"NN": {"Linears": [16, 32, 16, 8, 1],
                 "epoch_num": epoch_num,
                 "optimizer": "Adam",
                 "lr": 0.001,
                 "loss_type": "MSELoss",
                 "R2":{}}}
for target, target_name in zip(targets.T, target_names.keys()):
    print(f"######{target_name}######")
    for train_index, test_index in kf.split(features):
        x_train, y_train, x_test, y_test = datas_to_device(device, features[train_index, :], 
                                                            target[train_index],
                                                            features[test_index, :],
                                                            target[test_index]
                                                           )
        nn_model = netModel()
        nn_model.to(device)
        nn_model.double()
        criterion = nn.MSELoss()
        optimizer = optim.Adam(nn_model.parameters(), lr=0.001)
        for epoch in range(epoch_num):
            def closure():
                optimizer.zero_grad()
                out = nn_model(x_train)
                loss = criterion(torch.squeeze(out), torch.squeeze(y_train)).to(device)
                loss.backward()
                return loss
            
            optimizer.step(closure)
        pred_train = nn_model(x_train)
        pred = nn_model(x_test)
        r2_train = r2_score(torch.squeeze(y_train).cpu().detach().numpy(),
                                    torch.squeeze(pred_train).cpu().detach().numpy())
        r2_test = r2_score(torch.squeeze(y_test).cpu().detach().numpy(),
                                   torch.squeeze(pred).cpu().detach().numpy())
        print(r2_train)
        print(r2_test)
        if not result["NN"]["R2"].get(target_name, None):
            result["NN"]["R2"][target_name] = {"Train": [r2_train], "Test": [r2_test]}
        else:
            result["NN"]["R2"][target_name]["Train"].append(r2_train)
            result["NN"]["R2"][target_name]["Test"].append(r2_test)